In [11]:
import ETCE.ScenarioA.Ex04.weather_data as wd
import datetime
# DO NOT REMOVE THE ABOVE IMPORT.
# Additionally, you can import any python standard library

This function must accept a weather prediction of the form:

    {"uuid" : "some-uuid",    
    "timestamp": 00000000,
    "temp":5.33,
    "clouds":100.0,
    "wind_speed":2.48,
    "wind_deg":31.0,
    "wind_gust":4.1,
    "rain":1.54
    }
    
Return the predicted power output of the solar farm.

Solar farm Capacity:    50kW
Solar farm gives 0kW output from 9pm to 6am    

Solar Farm output is linear in cloudiness:
Clouds=0% => Solar Farm outputs at 100% Capacity
Clouds=100% => Solar Farm outputs at 10% Capacity

In [12]:
def predict_solar_output(weather_data):
    solar_capacity = 50  # kW
    total_solar_output = 0
    solar_output_by_timestamp = {}
    count = 0
    
    for timestamp, data in weather_data.items():
        time = datetime.datetime.fromtimestamp(timestamp)
        
        if 21 <= time.hour or time.hour < 6:
            power_output = 0
        else:
            
            clouds = data['clouds']
            output_percentage = 0.9 * (100 - clouds) / 100 + 0.1
            power_output = solar_capacity * output_percentage
        
        if count == 0:
            solar_output_by_timestamp[timestamp] = {"power_output": power_output, "energy_accumulated": 0}
        else:
            prev_energy = solar_output_by_timestamp[prev_timestamp]["energy_accumulated"]
            solar_output_by_timestamp[timestamp] = {"power_output": power_output, "energy_accumulated": prev_energy + power_output}

        total_solar_output += power_output
        prev_timestamp = timestamp
        count += 1

    average_output = total_solar_output / count if count > 0 else 0
    return solar_output_by_timestamp, average_output


This function must accept a weather prediction of the form:

    {"uuid" : "some-uuid",
    "timestamp": 00000000,
    "temp":5.33,
    "clouds":100.0,
    "wind_speed":2.48,
    "wind_deg":31.0,
    "wind_gust":4.1,
    "rain":1.54
    }
    
Return the predicted power output of the wind farm.

Wind farm Capacity: 50kW    
Wind Farm output % is linear in wind_speed

Wind Farm cut in speed = 3m/s
Wind Farm rated speed = 11m/s

In [13]:
def predict_wind_output(weather_data):
    wind_capacity = 50  
    total_wind_output = 0
    wind_output_by_timestamp = {}
    count = 0
    
    for timestamp, data in weather_data.items():
        wind_speed = data['wind_speed']
        
        if wind_speed < 3:  
            power_output = 0
        elif wind_speed >= 11:  
            power_output = wind_capacity
        else:  
            power_output = wind_capacity * (wind_speed - 3) / (11 - 3)
        
        if count == 0:
            wind_output_by_timestamp[timestamp] = {"power_output": power_output, "energy_accumulated": 0}
        else:
            prev_energy = wind_output_by_timestamp[prev_timestamp]["energy_accumulated"]
            wind_output_by_timestamp[timestamp] = {"power_output": power_output, "energy_accumulated": prev_energy + power_output}

        total_wind_output += power_output
        prev_timestamp = timestamp
        count += 1

    average_output = total_wind_output / count if count > 0 else 0
    return wind_output_by_timestamp, average_output


This function should return the solution to the Ex04

Return a tuple with the following structure:

    (({<integer timestamp>: 
     {"power_output":<float (kW)>,
      "energy_accumulated":<float kWh>},
     ...,
     }, <float average power output in %>),
    ({<integer timestamp>: 
     {"power_output":<float (kW)>,
      "energy_accumulated":<float kWh>},
     ...,
     }, <float average power output in %>))

power_output must correspond to the timestamp (result of a call to
solar() and wind()).

energy_accumulated must be total energy accumulated until that
timestamp (assumed to be 0 at the first timestamp)

The first and second nested tuples represent the power output of
the solar and wind farms respectively

In [14]:
def solution():
    weather_predictions = wd.get_predictions()
    

    start_time = datetime.datetime(2021, 4, 10, 2, 0, 0)
    start_timestamp = int(start_time.timestamp())
    
    filtered_data = {ts: data for ts, data in weather_predictions.items() if ts > start_timestamp}
    
    solar_results, avg_solar_output = predict_solar_output(filtered_data)
    wind_results, avg_wind_output = predict_wind_output(filtered_data)

    return (solar_results, avg_solar_output), (wind_results, avg_wind_output)


Evaluate your solution:

In [15]:
from driver import evaluate
wd.i = 0
evaluate(solution, solar, wind)

solution() failed to run:


InvalidOperation: [<class 'decimal.ConversionSyntax'>]